# Fine-Tuning a QA Model on Anime Fandom Data
This notebook fine-tunes a `flan-t5-base` model on a custom anime QA dataset stored in a CSV file.

In [9]:
import os
from datasets import load_dataset

path = "questions creation/data/qa_dataset.csv"

dataset = load_dataset("csv", data_files=path)
dataset = dataset["train"]
dataset = dataset.train_test_split(test_size=0.1)

Generating train split: 919 examples [00:00, 15442.60 examples/s]


In [10]:
from transformers import AutoTokenizer

model_checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess(example):
    input_text = f"question: {example['question']} context: {example['context']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(example['answer'], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=False)

Map: 100%|██████████| 92/92 [00:00<00:00, 450.94 examples/s]


In [3]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

training_args = TrainingArguments(
    output_dir="./anime_qa_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,  
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_dir="./logs",
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)], 
)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.139300,0.074668
2,0.065300,0.062979
3,0.068800,0.062656
4,0.023600,0.063105
5,0.064000,0.063502
6,0.019500,0.066538


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=1242, training_loss=1.1376354960715713, metrics={'train_runtime': 976.7146, 'train_samples_per_second': 16.934, 'train_steps_per_second': 4.239, 'total_flos': 3397765982846976.0, 'train_loss': 1.1376354960715713, 'epoch': 6.0})

In [11]:
from transformers import pipeline

base_pipe = pipeline("text2text-generation", model=model_checkpoint, tokenizer=model_checkpoint)
fine_pipe = pipeline("text2text-generation", model="./anime_qa_model/checkpoint-1242", tokenizer=model_checkpoint)

def truncate_prompt(prompt, tokenizer, max_length=512):
    tokens = tokenizer(prompt, truncation=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(tokens["input_ids"][0], skip_special_tokens=True)

def compare_models_on_chunk(question, context, tokenizer, max_output_tokens=150):
    prompt = f"Answer the question based on the text below:\n\n{context}\n\nQuestion: {question}\nAnswer:"
    prompt = truncate_prompt(prompt, tokenizer)

    input_length = tokenizer(prompt, return_tensors='pt')['input_ids'].shape[1]
    max_length = min(max_output_tokens, int(input_length * 0.5) + 20)

    print(f"\n Question:\n{question}")
    print(f"\n Context:\n{context[:500]}{'...' if len(context) > 500 else ''}")

    try:
        base_answer = base_pipe(prompt, max_length=max_length, do_sample=False)[0]['generated_text']
        fine_answer = fine_pipe(prompt, max_length=max_length, do_sample=False)[0]['generated_text']

        print(f"\n Base Model Answer:\n{base_answer}")
        print(f"\n Fine-Tuned Model Answer:\n{fine_answer}")

    except Exception as e:
        print(f"⚠️ Error: {e}")

Device set to use cuda:0
Device set to use cuda:0


In [12]:
context = """Omnipotence ArcWhen Boruto was called in for his first mission in a long time, Hinata cried, worried that this time he might not come back, despite his assurance he would return. After he left, Himawari told her she could help Boruto if she became a shinobi, and asked if it would make her worry more instead. Hinata slaps her adopted son. While preparing for dinner with Naruto, she wished that their sons, Boruto and Kawaki, could join them. She asked Naruto how long their mission might take. After leaving the house, Kawaki joined them, wanting to talk to Naruto. Kawaki expressed how much Naruto meant to him and how far he was willing to go to eliminate all the Ōtsutsuki, including Boruto. Hinata, hurt by hearing her own son say he would kill his brother, slapped him. Kawaki conceded that he might be insane for what he was willing to do, even if it meant killing his own brother. He then sent Naruto and Hinata away through a rift to prevent them from interfering, ready to bear their hatred and welcoming them to kill him when he was done. Naruto and Hinata were trapped inDaikokuten, suspended in time, unable to think, age, or require air or sustenance."""

question = "What was hinata worried about?"

compare_models_on_chunk(question, context, tokenizer)


 Question:
What was hinata worried about?

 Context:
Omnipotence ArcWhen Boruto was called in for his first mission in a long time, Hinata cried, worried that this time he might not come back, despite his assurance he would return. After he left, Himawari told her she could help Boruto if she became a shinobi, and asked if it would make her worry more instead. Hinata slaps her adopted son. While preparing for dinner with Naruto, she wished that their sons, Boruto and Kawaki, could join them. She asked Naruto how long their mission might take. Afte...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 Base Model Answer:
Boruto might not come back

 Fine-Tuned Model Answer:
this time he might not come back


In [13]:
context = """Killua has spiky white hair, very pale skin, and blue eyes. His eyes change shape depending on his mood, narrowing and sharpening when he goes into assassination mode. Killua is fairly lean at the start of the series, due to constant physical conditioning and torture training he received when he was young. During the Chimera Ant Arc, he becomes more muscular and toned. In the 1999 anime adaptation, Killua's eyes are green. He is also often seen holding a green skateboard (turned yellow in the 2011 anime adaptation). Killua typically wears baggy clothing, usually a dark-coloredturtleneck. In the manga and 2011 anime adaptation, Killua wears long, baggy shorts, but the 1999 anime adaptation character design shortened them to end above his knees. Killua also wears purple boots, recolored brown and black in the 1999 anime adaptation. Killua's hair was longer as a child, almost to his shoulders. He wore a hoodie with his trademark blue coloring, grey pants, and shoes."""

question = "What color is Killua Hair?"

compare_models_on_chunk(question, context, tokenizer)

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 Question:
What color is Killua Hair?

 Context:
Killua has spiky white hair, very pale skin, and blue eyes. His eyes change shape depending on his mood, narrowing and sharpening when he goes into assassination mode. Killua is fairly lean at the start of the series, due to constant physical conditioning and torture training he received when he was young. During the Chimera Ant Arc, he becomes more muscular and toned. In the 1999 anime adaptation, Killua's eyes are green. He is also often seen holding a green skateboard (turned yellow in the 20...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 Base Model Answer:
white

 Fine-Tuned Model Answer:
white


In [14]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline, AutoTokenizer
import torch

def scrape_fandom_page(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    title = soup.find('h1').text.strip()
    content_div = soup.find('div', {'class': 'mw-parser-output'})
    chunks, current_chunk, current_section_title = [], [], "Introduction"

    for tag in content_div.find_all(['h2', 'h3', 'p']):
        if tag.name in ['h2', 'h3']:
            if current_chunk:
                chunks.append({
                    "section": current_section_title,
                    "text": " ".join(current_chunk)
                })
                current_chunk = []
            current_section_title = tag.get_text(strip=True)
        elif tag.name == 'p':
            text = tag.get_text(strip=True)
            if text:
                current_chunk.append(text)

    if current_chunk:
        chunks.append({
            "section": current_section_title,
            "text": " ".join(current_chunk)
        })

    return {'title': title, 'url': url, 'chunks': chunks}

In [18]:
def compare_model_answers(question, retriever, top_k=3):
    top_chunks = retriever.query(question, top_k=top_k)
    context = "\n".join(chunk["text"] for chunk in top_chunks)

    prompt = f"Answer the question based on the text below:\n\n{context}\n\nQuestion: {question}\nAnswer:"
    prompt = truncate_prompt(prompt, tokenizer)

    input_tokens = tokenizer(prompt, return_tensors='pt')['input_ids'].shape[1]
    max_output_tokens = min(200, int(input_tokens * 0.5) + 20)

    base_answer = base_pipe(prompt, max_length=max_output_tokens, do_sample=False)[0]['generated_text']
    fine_answer = fine_pipe(prompt, max_length=max_output_tokens, do_sample=False)[0]['generated_text']

    print("\n Question:", question)
    print("\n Top Chunks Used:")
    for i, chunk in enumerate(top_chunks):
        print(f"\nChunk #{i+1} - Section: {chunk['section']}\n{chunk['text'][:300]}...\n")

    print("\n Base Model Answer:\n", base_answer)
    print("\n Fine-Tuned Model Answer:\n", fine_answer)

In [20]:
class Retriever:
    def __init__(self, chunks):
        self.chunks = chunks
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embeddings = self.model.encode([chunk['text'] for chunk in chunks], convert_to_tensor=True)

    def query(self, question, top_k=3):
        query_embedding = self.model.encode(question, convert_to_tensor=True)
        scores = cosine_similarity([query_embedding.cpu().numpy()], self.embeddings.cpu().numpy())[0]
        top_indices = np.argsort(scores)[::-1][:top_k]
        return [self.chunks[i] for i in top_indices]

In [21]:
url = "https://naruto.fandom.com/wiki/Hinata_Hyūga"
result = scrape_fandom_page(url)
retriever = Retriever(result['chunks'])

# Ask any question you'd like:
compare_model_answers("What are Hinata's special abilities?", retriever)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 Question: What are Hinata's special abilities?

 Top Chunks Used:

Chunk #1 - Section: Chakra Prowess and Control[]
Hinata's Gentle Step: Twin Lion Fist using Hamura's chakra. From her special clan training, Hinata has advanced control of her chakra. In the anime, Hinata's additional training during Part I resulted in her ability to free herself from contraints[29]and manipulate nearby water sources, turning them...


Chunk #2 - Section: Power[]
Main article:PowerIn the anime, Hinata is part of a team of reinforcements sent toTonika Villageto help Team 7 in the fight againstKabuto Yakushi. Hinata attempts to stop theNine-Tailed Naruto Clonefrom taking chakra from Naruto, but she fails and is saved by Neji....


Chunk #3 - Section: Video Games[]
Hinata Hyūga is a playable character in the following video games: In later instalments of theClash of Ninjaseries, Hinata is playable in an "awakened" form. In this form, her clothes are similar to what she wore during theBikōchū Search Missi